In [ ]:
import io
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath
import datetime
import psycopg2


In [ ]:
# Test code for cleaning data i.e. replace -ver values with NaN
# and Missing values imoputation by last observation carry forward scheme.
"""
df_mv_test.drop(df_mv_test.index ,inplace=True)
df_mv_test = pd.DataFrame()
df_mv_test['Sales'] = [np.nan,np.nan,6,-1,2,np.nan,np.nan,-5,8, np.nan]
df_mv_test['Accounts'] = [np.nan,np.nan,6,-1,2.0,np.nan,np.nan,-5,8, np.nan]
print(df_mv_test)
#df_mv_test = df_mv_test.where(df_mv_test > 0, np.nan) # one of the ways to replace -ve values with NaN
print(df_mv_test.mask(df_mv_test < 0)) 
#df.mask(df < 0, 0)  # to replace negative values with 0
print(df_mv_test.mask(df_mv_test < 0).ffill().bfill())
"""

To extract TEMPERATURE DATA for all sepsis patients that have time overlap for sepsis onset and shock onset timme OR sepsis onset time + 31 hours (all patietns with less and more than 20% missing values)

In [ ]:
df_csvdata = pd.read_csv('/Users/safiya.hashmi/MasterThesis/CSV files generated/Only_AllSepsisPatients_with_MissingData_FromSepsisOnset_ToShockOnset_or_SepsisOnset+31h.csv')
# Dataset is now stored in a Pandas Dataframe
print ('shape of original dataframe from CSV : ', df_csvdata.shape)


df_sepsisOnset_SepsisOnsetPlus31h_csvdata = df_csvdata[(df_csvdata['Sepsis_SepsisOnset+ShockOnsetOR31h_timeoverlap_exists']==1) ]

print ('shape of the dataframe after we select only icu stays that have all required signals and timeoverlap from CSV : ', df_sepsisOnset_SepsisOnsetPlus31h_csvdata.shape)

icustay_id_list = df_sepsisOnset_SepsisOnsetPlus31h_csvdata['icustay_id']
print('printing only ICU stay IDs to consider', len( icustay_id_list))



('shape of original dataframe from CSV : ', (2703, 13))
('shape of the dataframe after we select only icu stays that have all required signals and timeoverlap from CSV : ', (1056, 13))
('printing only ICU stay IDs to consider', 1056)


In [ ]:

# To connect to the mimic DB
import psycopg2
conn = psycopg2.connect(database="mimic",user="postgres",password="postgres",host="localhost", port="5432")
cur = conn.cursor()
cur.execute("SET search_path TO " + "mimiciii")


In [ ]:
import pandas.io.sql as psql
try:
  df_AllSepsisOnsetPlus31h_tempdata.drop(df_AllSepsisOnsetPlus31h_tempdata.index, inplace=True)
except:
  print('df_ALL1655_tempdata does not exist')
df_AllSepsisOnsetPlus31h_tempdata = pd.DataFrame(columns=['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','VALUE'])
for icustay_id in icustay_id_list:
  try:
    try:
      df_temp_result.drop(df_temp_result.index, inplace =True)
    except:
      print('df_temp_result does not exist')

    query_getTempData =  """select coh.subject_id, coh.icustay_id,ch.charttime,
       case
         when ch.itemid=223761 then
           (ch.valuenum -32)/1.8
       else
          ch.valuenum
       end as value
        from chartevents ch , sepsis3_cohort coh
        where ch.icustay_id = coh.icustay_id
        and ch.itemid in (223761,223762)
        and ch.error IS DISTINCT FROM 1
        and ch.icustay_id= """ + str(icustay_id) + "order by ch.charttime"; 

    cur.execute(query_getTempData);

    df_temp_result = pd.DataFrame(cur.fetchall(),columns=['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','VALUE'])

    #print(df_temp_result)

    df_AllSepsisOnsetPlus31h_tempdata = df_AllSepsisOnsetPlus31h_tempdata.append(df_temp_result,ignore_index=True)
    print('Appended TEMP data successfully for ICU STAY : ' ,icustay_id )
  
  except:
    print('Error occurred while appending TEMP data for ICU STAY : ' ,icustay_id )

#print(df_ALL1655_tempdata)
#df_ALL1655_tempdata.to_csv

df_ALL1655_tempdata does not exist
df_temp_result does not exist
('Appended TEMP data successfully for ICU STAY : ', 200217)
('Appended TEMP data successfully for ICU STAY : ', 200286)
('Appended TEMP data successfully for ICU STAY : ', 200325)
('Appended TEMP data successfully for ICU STAY : ', 200463)
('Appended TEMP data successfully for ICU STAY : ', 200526)
('Appended TEMP data successfully for ICU STAY : ', 200544)
('Appended TEMP data successfully for ICU STAY : ', 200773)
('Appended TEMP data successfully for ICU STAY : ', 200824)
('Appended TEMP data successfully for ICU STAY : ', 201089)
('Appended TEMP data successfully for ICU STAY : ', 201104)
('Appended TEMP data successfully for ICU STAY : ', 201271)
('Appended TEMP data successfully for ICU STAY : ', 201422)
('Appended TEMP data successfully for ICU STAY : ', 201679)
('Appended TEMP data successfully for ICU STAY : ', 201924)
('Appended TEMP data successfully for ICU STAY : ', 201970)
('Appended TEMP data successfully f

In [ ]:
print(df_AllSepsisOnsetPlus31h_tempdata.shape)

(50345, 4)


In [ ]:
df_AllSepsisOnsetPlus31h_tempdata.to_csv('df_All_1056_SepsisOnset_ShockOnset_OR_SepsisOnsetPlus31h_tempdata.csv', sep=',', index = False, header=True);



